<a href="https://colab.research.google.com/github/RishabhNegi1/Research-Fake-News-/blob/main/Adi_D4_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers datasets scikit-learn pandas torch emoji==0.6.0 optuna

import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import optuna
import pickle
from google.colab import files
import zipfile
import os


# Assuming the CSV files are named 'Fake.csv' and 'True.csv'
df_fake = pd.read_csv('Fake.csv')
df_real = pd.read_csv('True.csv')

# Add a 'label' column to each dataset
df_fake['label'] = 0  # Fake news
df_real['label'] = 1  # Real news

# Combine the datasets
df_combined = pd.concat([df_fake, df_real], ignore_index=True)

# Shuffle the dataset to mix fake and real news
df_combined = df_combined.sample(frac=1).reset_index(drop=True)

# Save the combined dataset to a new CSV file (optional)
df_combined.to_csv('combined_news.csv', index=False)

# Display the first few rows to understand the structure
print(df_combined.head())

# Inspect the column names to identify the text and label columns
print(df_combined.columns)

# Update these variables with the actual column names in your dataset
text_column = 'text'  # Assuming the text column is named 'text'
label_column = 'label'  # The correct column name for the labels

# Encode the labels as integers
label_encoder = LabelEncoder()
df_combined[label_column] = label_encoder.fit_transform(df_combined[label_column])

# Verify the encoding (optional)
print("Encoded labels:", label_encoder.classes_)

# Preprocess the dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load BERT base uncased tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df_combined[label_column].unique()))

# Split the dataset into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_combined[text_column].values,
    df_combined[label_column].values,
    test_size=0.2,
    random_state=42
)

# Define max token length
MAX_LEN = 128

# Create train and validation datasets
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, MAX_LEN)

# Use the best hyperparameters from the Optuna study
best_params = {
    'learning_rate': 1.6233212017995447e-05,
    'num_train_epochs': 3,
    'per_device_train_batch_size': 8
}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=best_params['num_train_epochs'],
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=best_params['learning_rate']
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the final model
trainer.train()

# Evaluate the final model
eval_result = trainer.evaluate()

# Predictions for validation data
predictions = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)

# Convert label classes to strings (necessary for classification_report)
target_names = [str(label) for label in label_encoder.classes_]

# Calculate and print detailed classification metrics
accuracy = accuracy_score(val_labels, preds)
conf_matrix = confusion_matrix(val_labels, preds)
class_report = classification_report(val_labels, preds, target_names=target_names)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

# Optional: Save the label encoder for later use
with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)


                                               title  \
0  The Secret Society That Ruined the World: Rhod...   
1  SELLING OUT AMERICA TO TERRORIST STATES IS HAR...   
2  U.S. completes 'takedown' of Medicare fraud: o...   
3  China has no problem overcoming middle-income ...   
4  Protesters throw rocks, bottles at police outs...   

                                                text       subject  \
0  IMAGE: Globalist scribe for the American branc...   Middle-east   
1   I know you ll miss me.  Did he really say tha...      politics   
2  WASHINGTON (Reuters) - U.S. law enforcement of...  politicsNews   
3  BEIJING (Reuters) - China has no problem overc...     worldnews   
4  (Reuters) - Protesters threw rocks and bottles...  politicsNews   

                 date  label  
0  September 20, 2017      0  
1         Aug 8, 2015      0  
2      June 22, 2016       1  
3   October 19, 2017       1  
4       May 25, 2016       1  
Index(['title', 'text', 'subject', 'date', 'label'], dty

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.000000,0.004593
2,0.000000,0.002692
3,0.000000,0.000604


Accuracy: 0.9998886414253898
Confusion Matrix:
 [[4732    0]
 [   1 4247]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4732
           1       1.00      1.00      1.00      4248

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

